<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighbourhoods in Toronto</font></h1>

## Introduction

In this notebook, we will:
 <ul>
    <li>Convert addresses into their equivalent latitude and longitude values.
    <li>Use the Foursquare API to explore neighbourhoods in Toronto.
    <li>Use the **explore** function to get the most common venue categories in each neighbourhood.
    <li>Use this feature to group the neighbourhoods into clusters.
    <li>Use the *k*-means clustering algorithm to complete this task.
    <li>Use the Folium library to visualize the neighbourhoods in Toronto and their emerging clusters.
 </ul>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Explore Dataset</a>

2. <a href="#item2">Explore Neighbourhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighbourhood</a>

4. <a href="#item4">Cluster Neighbourhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Explore Dataset

#### Load and explore the data as a *pandas* dataframe

Let's load the data.

In [2]:
# [T]oronto [N]eighbourhoods [D]ata
TND =\
    pd\
        .read_csv("../data_files/toronto_neighbourhoods_coordinates_data.csv")\
        .set_index("CDN Number")\
        .sort_index(ascending=True)

Let's take a look at the `TND` dataframe to confirm that the columns are as intended.

In [3]:
TND.head()

,Neighbourhood,Borough,Latitude,Longitude
CDN Number,,,,
1,West Humber-Clairville,Etobicoke,43.71456,-79.59261
2,Mount Olive-Silverstone-Jamestown,Etobicoke,43.74721,-79.58826
3,Thistletown-Beaumond Heights,Etobicoke,43.73858,-79.56428
4,Rexdale-Kipling,Etobicoke,43.72433,-79.56720
5,Elms-Old Rexdale,Etobicoke,43.72429,-79.54956


And make sure that the dataset has all 7 boroughs and 140 neighbourhoods.

In [4]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(TND['Borough'].unique()),
        TND.shape[0]
    )
)

The dataframe has 7 boroughs and 140 neighbourhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [5]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighbourhoods superimposed on top.

In [6]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(TND['Latitude'], TND['Longitude'], TND['Borough'], TND['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        (lat, lng),
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
    
toronto_map

<a id='item2'></a>

## 2. Explore Neighbourhoods in Toronto

We are going to start utilizing the Foursquare API to explore the neighbourhoods and segment them.

#### Define Foursquare Credentials and `VERSION`

In [7]:
CLIENT_ID = 'your-client-id' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-id
CLIENT_SECRET: your-client-secret


#### Define `LIMIT` and `RADIUS`

In [8]:
LIMIT = 100
RADIUS = 500

#### Let's create a function to get recommended venues for all the neighbourhoods in Toronto

In [9]:
from time import sleep

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            RADIUS,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        sleep(1)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighbourhood Latitude',
                  'Neighbourhood Longitude',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighbourhood and create a new dataframe called *toronto_venues*.

In [10]:
toronto_venues = getNearbyVenues(
    names=TND['Neighbourhood'],
    latitudes=TND['Latitude'],
    longitudes=TND['Longitude']
)

West Humber-Clairville
Mount Olive-Silverstone-Jamestown
Thistletown-Beaumond Heights
Rexdale-Kipling
Elms-Old Rexdale
Kingsview Village-The Westway
Willowridge-Martingrove-Richview
Humber Heights-Westmount
Edenbridge-Humber Valley
Princess-Rosethorn
Eringate-Centennial-West Deane
Markland Wood
Etobicoke West Mall
Islington-City Centre West
Kingsway South
Stonegate-Queensway
Mimico
New Toronto
Long Branch
Alderwood
Humber Summit
Humbermede
Pelmo Park-Humberlea
Black Creek
Glenfield-Jane Heights
Downsview-Roding-CFB
York University Heights
Rustic
Maple Leaf
Brookhaven-Amesbury
Yorkdale-Glen Park
Englemount-Lawrence
Clanton Park
Bathurst Manor
Westminster-Branson
Newtonbrook West
Willowdale West
Lansing-Westgate
Bedford Park-Nortown
St. Andrew-Windfields
Bridle Path-Sunnybrook-York Mills
Banbury-Don Mills
Victoria Village
Flemingdon Park
Parkwoods-Donalda
Pleasant View
Don Valley Village
Hillcrest Village
Bayview Woods-Steeles
Newtonbrook East
Willowdale East
Bayview Village
Henry Farm
O

#### Let's check the size of the resulting dataframe

In [11]:
print(toronto_venues.shape)
toronto_venues.head()

(2007, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,West Humber-Clairville,43.71456,-79.59261,Tim Hortons,43.714657,-79.593716,Coffee Shop
1,West Humber-Clairville,43.71456,-79.59261,Xawaash,43.715786,-79.593053,Mediterranean Restaurant
2,West Humber-Clairville,43.71456,-79.59261,Staples Rexdale,43.718539,-79.594570,Paper / Office Supplies Store
3,West Humber-Clairville,43.71456,-79.59261,Swiss Pick,43.716150,-79.593843,Swiss Restaurant
4,West Humber-Clairville,43.71456,-79.59261,Woodbine Hotel & Suites,43.712624,-79.592664,Hotel


Let's check how many venues were returned for each neighbourhood

In [12]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt North,28,28,28,28,28,28
Agincourt South-Malvern West,22,22,22,22,22,22
Alderwood,4,4,4,4,4,4
Annex,54,54,54,54,54,54
Banbury-Don Mills,5,5,5,5,5,5
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,65,65,65,65,65,65
Bayview Village,4,4,4,4,4,4
Bayview Woods-Steeles,2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues

In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighbourhood

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# move Neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [15]:
toronto_onehot.shape

(2007, 272)

#### Next, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [16]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Stadium,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

#### Let's confirm the new size

In [17]:
toronto_grouped.shape

(139, 272)

#### Let's print each neighbourhood along with the top 5 most common venues

In [18]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North----
                venue  freq
0                Bank  0.07
1              Bakery  0.07
2  Chinese Restaurant  0.07
3   Convenience Store  0.04
4          Beer Store  0.04


----Agincourt South-Malvern West----
                      venue  freq
0        Chinese Restaurant  0.27
1      Cantonese Restaurant  0.09
2  Mediterranean Restaurant  0.05
3          Asian Restaurant  0.05
4           Motorcycle Shop  0.05


----Alderwood----
               venue  freq
0  Convenience Store  0.25
1        Coffee Shop  0.25
2           Pharmacy  0.25
3        Pizza Place  0.25
4          Pet Store  0.00


----Annex----
                venue  freq
0              Bakery  0.07
1                Café  0.07
2  Italian Restaurant  0.06
3      Sandwich Place  0.06
4          Restaurant  0.06


----Banbury-Don Mills----
          venue  freq
0  Liquor Store   0.2
1   Coffee Shop   0.2
2      Pharmacy   0.2
3           Spa   0.2
4          Café   0.2


----Bathurst Manor----
              

                       venue  freq
0             Discount Store  0.25
1         African Restaurant  0.25
2  Latin American Restaurant  0.25
3              Grocery Store  0.25
4            Motorcycle Shop  0.00


----Humewood-Cedarvale----
               venue  freq
0       Hockey Arena   0.2
1              Trail   0.2
2              Field   0.2
3  Convenience Store   0.2
4     Soccer Stadium   0.2


----Ionview----
            venue  freq
0     Coffee Shop  0.12
1   Train Station  0.06
2   Metro Station  0.06
3        Pharmacy  0.06
4  Sandwich Place  0.06


----Islington-City Centre West----
                  venue  freq
0  Fast Food Restaurant  0.14
1           Coffee Shop  0.09
2            Restaurant  0.09
3           Pizza Place  0.09
4        Sandwich Place  0.09


----Junction Area----
           venue  freq
0    Coffee Shop  0.10
1     Restaurant  0.04
2  Grocery Store  0.04
3     Nail Salon  0.04
4            Bar  0.04


----Keelesdale-Eglinton West----
                       

               venue  freq
0               Park   0.5
1       Food Service   0.5
2  Accessories Store   0.0
3  Mobile Phone Shop   0.0
4             Museum   0.0


----Waterfront Communities-The Island----
            venue  freq
0     Coffee Shop  0.19
1      Restaurant  0.06
2    Liquor Store  0.06
3  Clothing Store  0.06
4           Beach  0.06


----West Hill----
                        venue  freq
0  Construction & Landscaping  0.29
1                        Park  0.29
2                    Tea Room  0.14
3           Convenience Store  0.14
4                  Restaurant  0.14


----West Humber-Clairville----
                           venue  freq
0                          Hotel  0.29
1       Mediterranean Restaurant  0.14
2                    Coffee Shop  0.14
3            Rental Car Location  0.14
4  Paper / Office Supplies Store  0.14


----Westminster-Branson----
           venue  freq
0    Pizza Place  0.12
1  Grocery Store  0.12
2       Pharmacy  0.12
3        Butcher  0.12
4 

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighbourhood.

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bank,Chinese Restaurant,Bakery,Liquor Store,Frozen Yogurt Shop,Fried Chicken Joint,Beer Store,Japanese Restaurant,Sporting Goods Shop,Spa
1,Agincourt South-Malvern West,Chinese Restaurant,Cantonese Restaurant,Pool,Pizza Place,Seafood Restaurant,Mediterranean Restaurant,Restaurant,Motorcycle Shop,Pool Hall,Bank
2,Alderwood,Coffee Shop,Pharmacy,Pizza Place,Convenience Store,Eastern European Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
3,Annex,Bakery,Café,Sandwich Place,Italian Restaurant,Restaurant,Beer Bar,Bookstore,Japanese Restaurant,Bar,Vegetarian / Vegan Restaurant
4,Banbury-Don Mills,Coffee Shop,Pharmacy,Café,Spa,Liquor Store,Egyptian Restaurant,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant


<a id='item4'></a>

## 4. Cluster Neighbourhoods

Run *k*-means to cluster the neighbourhoods into 5 clusters.

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood.

In [22]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = TND

# merge toronto_grouped with TND to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,,,,,
1,West Humber-Clairville,Etobicoke,43.71456,-79.59261,0.0,Hotel,Rental Car Location,Mediterranean Restaurant,Swiss Restaurant,Coffee Shop,Paper / Office Supplies Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Electronics Store
2,Mount Olive-Silverstone-Jamestown,Etobicoke,43.74721,-79.58826,2.0,Park,Coffee Shop,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
3,Thistletown-Beaumond Heights,Etobicoke,43.73858,-79.56428,0.0,Indian Restaurant,Caribbean Restaurant,Pizza Place,Supermarket,Coffee Shop,Bank,Pharmacy,Ice Cream Shop,Electronics Store,Egyptian Restaurant
4,Rexdale-Kipling,Etobicoke,43.72433,-79.56720,0.0,Lawyer,Convenience Store,Movie Theater,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
5,Elms-Old Rexdale,Etobicoke,43.72429,-79.54956,4.0,Construction & Landscaping,Golf Course,Business Service,Yoga Studio,Dumpling Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant


Finally, let's visualize the resulting clusters

In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    cluster = int(0 if np.isnan(cluster) else cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        (lat, lon),
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,
1,Etobicoke,Hotel,Rental Car Location,Mediterranean Restaurant,Swiss Restaurant,Coffee Shop,Paper / Office Supplies Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Electronics Store
3,Etobicoke,Indian Restaurant,Caribbean Restaurant,Pizza Place,Supermarket,Coffee Shop,Bank,Pharmacy,Ice Cream Shop,Electronics Store,Egyptian Restaurant
4,Etobicoke,Lawyer,Convenience Store,Movie Theater,Yoga Studio,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
6,Etobicoke,Park,Shopping Mall,Coffee Shop,Bus Line,Yoga Studio,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop
7,Etobicoke,Baseball Field,Park,Pharmacy,Gym,Moving Target,Bus Stop,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office
9,Etobicoke,Fast Food Restaurant,Indian Restaurant,Park,Bus Stop,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Yoga Studio
11,Etobicoke,Coffee Shop,Park,Convenience Store,Pizza Place,Chinese Restaurant,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop
13,Etobicoke,Skating Rink,Hotel,Farmers Market,Café,Sandwich Place,Coffee Shop,Convenience Store,Event Space,Ethiopian Restaurant,Electronics Store
14,Etobicoke,Fast Food Restaurant,Coffee Shop,Pizza Place,Restaurant,Sandwich Place,Greek Restaurant,Ice Cream Shop,Fried Chicken Joint,Bank,Thai Restaurant


#### Cluster 2

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,
12,Etobicoke,Park,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
15,Etobicoke,Park,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
36,North York,Park,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
40,North York,Park,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


#### Cluster 3

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,
2,Etobicoke,Park,Coffee Shop,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
8,Etobicoke,Gas Station,Pizza Place,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
16,Etobicoke,Park,Playground,Yoga Studio,Eastern European Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant
25,North York,Park,Tea Room,Plaza,Pool,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar,Doctor's Office
34,North York,Baseball Field,Park,Convenience Store,Playground,Yoga Studio,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop
37,North York,Park,Mobile Phone Shop,Yoga Studio,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Egyptian Restaurant
38,North York,Health & Beauty Service,Home Service,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
43,North York,Food Service,Park,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
48,North York,Residential Building (Apartment / Condo),Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant


#### Cluster 4

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,
69,Old City of Toronto,Toy / Game Store,Café,Yoga Studio,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 5

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
CDN Number,,,,,,,,,,,
5,Etobicoke,Construction & Landscaping,Golf Course,Business Service,Yoga Studio,Dumpling Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant
10,Etobicoke,Construction & Landscaping,Yoga Studio,Egyptian Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
21,North York,Construction & Landscaping,Rental Car Location,Music Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop


### Thank you for your time!

This notebook was created by **Mohammed Madbouli**

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2020 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).